In [1]:
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
import re
from bs4 import BeautifulSoup

In [2]:
URL = 'https://yearn.finance/vaults'
# page = requests.get(URL)

# initiating the webdriver. Parameter includes the path of the webdriver.
## make sure you keep chromedriver in the same path as the code
## install driver as per the chrome version
# driver = webdriver.Chrome('./chromedriver') 

driver_path = "./chromedriver"
brave_path = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"

option = webdriver.ChromeOptions()
option.binary_location = brave_path
# option.add_argument("--incognito") OPTIONAL
# option.add_argument("--headless") OPTIONAL

# Create new Instance of Chrome
driver = webdriver.Chrome(executable_path=driver_path, options=option)

driver.get(URL) 

# this is just to ensure that the page is loaded
time.sleep(5)

In [3]:
html = driver.page_source
  
# this renders the JS code and stores all
# of the information in static HTML code.
  
# Now, we could simply apply bs4 to html variable
soup = BeautifulSoup(html, "html.parser")

## Check the extracted data
print(soup.prettify())

<html lang="en">
 <head>
  <style>
   .error_widget_wrapper {        background: inherit;        color: inherit;        border:none    }    .error_widget {        border-top: solid 2px;        border-bottom: solid 2px;        margin: 5px 0;        padding: 10px 40px;        white-space: pre-wrap;    }    .error_widget.ace_error, .error_widget_arrow.ace_error{        border-color: #ff5a5a    }    .error_widget.ace_warning, .error_widget_arrow.ace_warning{        border-color: #F1D817    }    .error_widget.ace_info, .error_widget_arrow.ace_info{        border-color: #5a5a5a    }    .error_widget.ace_ok, .error_widget_arrow.ace_ok{        border-color: #5aaa5a    }    .error_widget_arrow {        position: absolute;        border: solid 5px;        border-top-color: transparent!important;        border-right-color: transparent!important;        border-left-color: transparent!important;        top: -5px;    }
  </style>
  <style id="ace-tm">
   .ace-tm .ace_gutter {background: #f0f0f0;colo

In [4]:
## Extract vault ids for all yearn vaults
all_ids = soup.findAll('div', id=re.compile('^vault-'))

## get all the vault ids
vault_ids = []
for vault_id in all_ids:
    vault_ids.append(vault_id.get('id'))

## check all the vault ids
print(vault_ids)

['vault-0xbD17B1ce622d73bD438b9E658acA5996dc394b0d', 'vault-0xc5bDdf9843308380375a611c18B50Fb9341f502A', 'vault-0x9d409a0A012CFbA9B15F6D4B36Ac57A46966Ab9a', 'vault-0xc695f73c1862e050059367B2E64489E66c525983', 'vault-0x5f18C75AbDAe578b483E5F43f12a39cF75b973a9', 'vault-0xA696a63cc78DfFa1a63E9E50587C197387FF6C7E', 'vault-0xa5cA62D95D24A4a350983D5B8ac4EB8638887396', 'vault-0x27b7b1ad7288079A66d12350c828D3C00A6F07d7', 'vault-0x8ee57c05741aA9DB947A744E713C15d4d19D8822', 'vault-0x8cc94ccd0f3841a468184aCA3Cc478D2148E1757', 'vault-0x23D3D0f1c697247d5e0a9efB37d8b0ED0C464f7f', 'vault-0x8fA3A9ecd9EFb07A8CE90A6eb014CF3c0E3B32Ef', 'vault-0xD6Ea40597Be05c201845c0bFd2e96A60bACde267', 'vault-0xFBEB78a723b8087fD2ea7Ef1afEc93d35E8Bed42', 'vault-0xB4AdA607B9d6b2c9Ee07A275e9616B84AC560139', 'vault-0x5fA5B62c8AF877CB37031e0a3B2f34A78e3C56A6', 'vault-0x625b7DF2fa8aBe21B0A976736CDa4775523aeD1E', 'vault-0x30FCf7c6cDfC46eC237783D94Fc78553E79d4E9C', 'vault-0xf8768814b88281DE4F532a3beEfA5b85B69b9324', 'vault-0xe9

In [5]:
import os

if os.path.isfile('yearn_vaults_data.csv'):
    yearn_data = pd.read_csv('yearn_vaults_data.csv', index_col=0)
else:
    yearn_data = pd.DataFrame(columns = ['vault_name', 'vault_type', 'vault_apy', 'vault_total_assets'], index=pd.to_datetime([]))

In [6]:
## extract the data for all vault ids

for vault_id in vault_ids:

    ## extract the vault data
    results = soup.find(id=vault_id)

    ## check all the extracted data
    # print(results.prettify())

    ## extract vault information using div class "Box__StyledBox-fqizq9-0 kZrUgR Text__StyledBox-sc-1ucgi3j-0 lbqtCv"
    results_value = results.find_all("div", {"class": "Box__StyledBox-fqizq9-0 kZrUgR Text__StyledBox-sc-1ucgi3j-0 lbqtCv"})

    ## check all the vault information
    for i, res in enumerate(results_value):
#         print(res, res.text)
        if i == 0:
            vault_name = res.text
        elif i == 1:
            vault_type = res.text
        elif i == 3:
            vault_apy = res.text
        elif i == 4:
            vault_total_assets = res.text
    print(vault_name, vault_type, vault_apy, vault_total_assets)
    
    ## storing data in pandas dataframe, timezone is in UTC
    yearn_data.loc[pd.to_datetime("now"), :] = (vault_name, vault_type, vault_apy, vault_total_assets)

yveCRV - ETH v1 N/A $0
yveCRV v1 46.61% N/A
yvBOOST v2 NEW ✨ $7,037,719
yvBOOST - ETH v2 NEW ✨ $0
USDC v2 14.67% $254,002,100
WBTC v2 1.59% $290,903,660
sUSD v2 NEW ✨ $809,111
crvIB v2 15.00% $223,276,039
crvYBUSD v2 12.97% $21,269,218
crvMUSD v2 15.83% $887,308
crvTBTC v2 7.57% $553
crvBBTC v2 -0.10% $10,166
crvCOMP v2 7.72% $33,682,107
UNI v2 0.73% $1,523,784
crvFRAX v2 7.29% $30,313,004
crvLUSD v2 22.56% $59,306,553
crvHBTC v2 8.94% $16,484,419
crvDUSD v2 24.94% $1,928,150
crvTUSD v2 17.46% $1,998
crvOBTC v2 8.55% $8,996,541
crvRENBTC v2 4.40% $112,983
crvSAAVE v2 9.75% $5,924,785
USDT v2 17.15% $17,951,560
DAI v2 12.18% $530,006,199
1INCH v2 20.97% $489,781
crvPBTC v2 14.52% $1,751,162
WETH v2 2.62% $143,552,215
yCRV v2 9.90% $24,333,115
crvBUSD v2 21.70% $6,554,457
crvSETH v2 8.30% $103,241,133
crvSTETH v2 9.41% $279,784,494
3Crv v2 5.70% $7,733,191
SNX v2 47.39% $2,861,391
crvUST v2 26.18% $4,630,032
YFI v2 1.08% $89,296,727
crvGUSD v2 7.12% $16,157,420
crvSBTC v2 2.87% $7,271,62

In [7]:
yearn_data.tail()

,vault_name,vault_type,vault_apy,vault_total_assets
2021-05-22 16:47:36.056954,TUSD,v1,9.09%,"$3,340,469"
2021-05-22 16:47:36.082218,DAI,v1,7.60%,"$7,094,862"
2021-05-22 16:47:36.112653,crvUSDP,v1,26.96%,"$61,315,293"
2021-05-22 16:47:36.140254,crvAETH,v1,9.95%,"$6,486,515"
2021-05-22 16:47:36.167203,mUSD,v1,-0.51%,"$11,507"


In [8]:
yearn_data.isna().sum()

vault_name            0
vault_type            0
vault_apy             0
vault_total_assets    0
dtype: int64

In [9]:
yearn_data[yearn_data['vault_total_assets'].isna()]

,vault_name,vault_type,vault_apy,vault_total_assets


In [10]:
yearn_data[yearn_data['vault_apy'].isna()]

,vault_name,vault_type,vault_apy,vault_total_assets


In [11]:
yearn_data['vault_apy'] = yearn_data['vault_apy'].fillna('0%')
yearn_data[yearn_data['vault_apy'].isna()]

,vault_name,vault_type,vault_apy,vault_total_assets


In [12]:
yearn_data['vault_total_assets'] = yearn_data['vault_total_assets'].fillna('$0')
yearn_data[yearn_data['vault_total_assets'].isna()]

,vault_name,vault_type,vault_apy,vault_total_assets


In [13]:
yearn_data[yearn_data['vault_total_assets']=='N/A']

,vault_name,vault_type,vault_apy,vault_total_assets
2021-05-22 16:47:35.176105,yveCRV,v1,46.61%,N/A


In [14]:
yearn_data[yearn_data['vault_apy']=='N/A']

,vault_name,vault_type,vault_apy,vault_total_assets
2021-05-22 16:47:35.167537,yveCRV - ETH,v1,N/A,$0


In [15]:
def convert_na_assets(x):
    if x == 'N/A':
        return '$0'
    return x

yearn_data['vault_total_assets'] = yearn_data['vault_total_assets'].apply(lambda x: convert_na_assets(x))
yearn_data[yearn_data['vault_total_assets']=='N/A']

,vault_name,vault_type,vault_apy,vault_total_assets


In [16]:
def convert_na_apy(x):
    if x == 'N/A':
        return '0.00%'
    return x

yearn_data['vault_apy'] = yearn_data['vault_apy'].apply(lambda x: convert_na_apy(x))
yearn_data[yearn_data['vault_apy']=='N/A']

,vault_name,vault_type,vault_apy,vault_total_assets


In [17]:
yearn_data.to_csv('yearn_vaults_data.csv')

In [18]:
yearn_data.head()

,vault_name,vault_type,vault_apy,vault_total_assets
2021-04-19 04:38:37.925270,yveCRV - ETH,v1,0%,"$12,366,520"
2021-04-19 04:38:37.932958,yveCRV,v1,19.16%,$0
2021-04-19 04:38:37.937337,USDC,v2,16.48%,"$125,785,880"
2021-04-19 04:38:37.942564,crvIB,v2,30.02%,"$135,500,556"
2021-04-19 04:38:37.947768,HEGIC,v2,0.77%,"$708,539"


In [19]:
yearn_data.shape

(1566, 4)

### Extra Code Below ----------------------------------- not needed -----------------------

In [40]:
## extract the data for yveCRV - ETH value with address "vault-0xbD17B1ce622d73bD438b9E658acA5996dc394b0d"

## extract the vault data
results = soup.find(id='vault-0xbD17B1ce622d73bD438b9E658acA5996dc394b0d')

## check all the extracted data
# print(results.prettify())

## extract vault information using div class "Box__StyledBox-fqizq9-0 kZrUgR Text__StyledBox-sc-1ucgi3j-0 lbqtCv"
results_value = results.find_all("div", {"class": "Box__StyledBox-fqizq9-0 kZrUgR Text__StyledBox-sc-1ucgi3j-0 lbqtCv"})

## check all the vault information
for i, res in enumerate(results_value):
#     print(res, res.text)
    if i == 0:
        vault_name = res.text
    elif i == 1:
        vault_type = res.text
    elif i == 3:
        vault_apy = res.text
    elif i == 4:
        vault_total_assets = res.text
print(vault_name, vault_type, vault_apy, vault_total_assets)

## storing data in pandas dataframe, timezone is in UTC
yearn_data.loc[pd.to_datetime("now"), :] = (vault_name, vault_type, vault_apy, vault_total_assets)

yveCRV - ETH v1 0.0000 53.07%


In [41]:
yearn_data.tail()

,vault_name,vault_type,vault_apy,vault_total_assets
2021-04-18 12:20:24.099055,yveCRV - ETH,v1,0.0000,53.07%
